# Deep Q-Learning 

For this assignment we will implement the Deep Q-Learning algorithm with Experience Replay as described in breakthrough paper __"Playing Atari with Deep Reinforcement Learning"__. We will train an agent to play the famous game of __Breakout__.

In [1]:
import sys
import gym
import torch
import pylab
import random
import numpy as np
import time
from collections import deque
from datetime import datetime
from copy import deepcopy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from utils import *
from agent import *
from model import *
from config import *
from env import GameEnv
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Understanding the environment

In the following cell, we initialise our game of __Breakout__ and you can see how the environment looks like. For further documentation of the of the environment refer to https://gym.openai.com/envs. 

In [2]:
envs = []
for i in range(num_envs):
    envs.append(GameEnv('SpaceInvadersDeterministic-v4'))
#env.render()

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [3]:
number_lives = envs[0].life
state_size = envs[0].observation_space.shape
action_size = envs[0].action_space.n
rewards, episodes = [], []

## Creating a DQN Agent

Here we create a DQN Agent. This agent is defined in the __agent.py__. The corresponding neural network is defined in the __model.py__. 

__Evaluation Reward__ : The average reward received in the past 100 episodes/games.

__Frame__ : Number of frames processed in total.

__Memory Size__ : The current size of the replay memory.

In [4]:
agent = Agent(action_size)
torch.save(agent.policy_net.state_dict(), "./save_model/spaceinvaders_ppo_best")
evaluation_reward = deque(maxlen=evaluation_reward_length)
frame = 0
memory_size = 0
reset_max = 10

### Main Training Loop

In [2]:
vis_env_idx = 0
vis_env = envs[vis_env_idx]
e = 0
frame = 0
max_eval = -np.inf
reset_count = 0

while (frame < 10000000):
    step = 0
    assert(num_envs * env_mem_size == train_frame)
    frame_next_vals = []
    for i in range(num_envs):
        env = envs[i]
        #history = env.history
        #life = env.life
        #state, reward, done, info = [env.state, env.reward, env.done, env.info]
        for j in range(env_mem_size):
            step += 1
            frame += 1
            
            curr_state = env.history[HISTORY_SIZE-1,:,:]
            action, value = agent.get_action(np.float32(env.history[:HISTORY_SIZE,:,:]) / 255.)
            
            next_state, env.reward, env.done, env.info = env.step(action)
            
            if (i == vis_env_idx):
                vis_env._env.render()
            
            frame_next_state = get_frame(next_state)
            env.history[HISTORY_SIZE,:,:] = frame_next_state
            terminal_state = check_live(env.life, env.info['ale.lives'])
            
            env.life = env.info['ale.lives']
            r = env.reward
            
            agent.memory.push(i, deepcopy(curr_state), action, r, terminal_state, value, 0, 0)
            if (j == env_mem_size-1):
                _, frame_next_val = agent.get_action(np.float32(env.history[1:,:,:]) / 255.)
                frame_next_vals.append(frame_next_val)
            env.score += r
            env.history[:HISTORY_SIZE, :, :] = env.history[1:,:,:]
            
            if (env.done):
                if (e % 50 == 0):
                    print('now time : ', datetime.now())
                    rewards.append(np.mean(evaluation_reward))
                    episodes.append(e)
                    pylab.plot(episodes, rewards, 'b')
                    pylab.savefig("./save_graph/spaceinvaders_ppo.png")
                    torch.save(agent.policy_net, "./save_model/spaceinvaders_ppo")
                    
                    if np.mean(evaluation_reward) > max_eval:
                        torch.save(agent.policy_net.state_dict(), "./save_model/spaceinvaders_ppo_best")
                        max_eval = float(np.mean(evaluation_reward))
                        reset_count = 0
                    elif e > 5000:
                        reset_count += 1
                        """
                        if (reset_count == reset_max):
                            print("Training went nowhere, starting again at best model")
                            agent.policy_net.load_state_dict(torch.load("./save_model/spaceinvaders_ppo_best"))
                            agent.update_target_net()
                            reset_count = 0
                        """
                e += 1
                evaluation_reward.append(env.score)
                print("episode:", e, "  score:", env.score,  " epsilon:", agent.epsilon, "   steps:", step,
                  " evaluation reward:", np.mean(evaluation_reward))
                
                env.done = False
                env.score = 0
                env.history = np.zeros([HISTORY_SIZE+1,84,84], dtype=np.uint8)
                env.state = env.reset()
                env.life = number_lives
                get_init_state(env.history, env.state)
                
                
                
    agent.train_policy_net(frame, frame_next_vals)
    agent.update_target_net()


NameError: name 'envs' is not defined

In [ ]:
torch.save(agent.policy_net, "./save_model/spaceinvaders_ppo")

In [ ]:
### Loop through all environments and run PPO on them

env_names = ['Breakout-v0', 'Phoenix-v0', 'Asteroids-v0', 'SpaceInvaders-v0', 'MsPacman-v0', 'Asterix-v0', 'Atlantis-v0', 'Alien-v0', 'Amidar-v0', 'Assault-v0', 'BankHeist-v0']
for a in range(len(env_names)):
    name = env_names[a]
    print("\n\n\n ------- STARTING TRAINING FOR %s ------- \n\n\n" % (name))
    
    envs = []
    for i in range(num_envs):
        envs.append(GameEnv(name))
    #env.render()
    

    number_lives = envs[0].life
    state_size = envs[0].observation_space.shape
    if (name == 'SpaceInvaders-v0' or name == 'Breakout-v0'):
        action_size = 4
    else:
        action_size = envs[0].action_space.n
    rewards, episodes = [], []

    vis_env_idx = 0
    vis_env = envs[vis_env_idx]
    e = 0
    frame = 0
    max_eval = -np.inf
    reset_count = 0


    agent = Agent(action_size)
    torch.save(agent.policy_net.state_dict(), "./save_model/" + name + "_best")
    evaluation_reward = deque(maxlen=evaluation_reward_length)
    frame = 0
    memory_size = 0
    reset_max = 10
    
    print("Determing min/max rewards of environment")
    [low, high] = score_range = get_score_range(name)
    print("Min: %d. Max: %d." % (low, high))

    while (frame < 10000000):
        step = 0
        assert(num_envs * env_mem_size == train_frame)
        frame_next_vals = []
        
        for j in range(env_mem_size):
            
            curr_states = np.stack([envs[i].history[HISTORY_SIZE-1,:,:] for i in range(num_envs)])
            next_states = []
            net_in = np.stack([envs[i].history[:HISTORY_SIZE,:,:] for i in range(num_envs)])
            step += num_envs
            frame += num_envs
            actions, values = agent.get_action(np.float32(net_in) / 255.)
            
            for i in range(num_envs):
                env = envs[i]
                next_state, env.reward, env.done, env.info = env.step(actions[i])
                next_states.append(next_state)
                if (i == vis_env_idx):
                    vis_env._env.render()
            
            for i in range(num_envs):
                env = envs[i]
                """
                next_state, env.reward, env.done, env.info = env.step(actions[i])
                if (i == vis_env_idx):
                    vis_env._env.render()
                """
                
                frame_next_state = get_frame(next_states[i])
                env.history[HISTORY_SIZE,:,:] = frame_next_state
                terminal_state = check_live(env.life, env.info['ale.lives'])
                env.life = env.info['ale.lives']
                r = (env.reward / high) * 20.0 #np.log(max(env.reward+1, 1))#((env.reward - low) / (high - low)) * 30
                agent.memory.push(i, deepcopy(curr_states[i]), actions[i], r, terminal_state, values[i], 0, 0)
                
                if (j == env_mem_size-1):
                    net_in = np.stack([envs[k].history[1:,:,:] for k in range(num_envs)])
                    _, frame_next_vals = agent.get_action(np.float32(net_in) / 255.)
                
                env.score += env.reward
                env.history[:HISTORY_SIZE, :, :] = env.history[1:,:,:]
        
                if (env.done):
                    if (e % 50 == 0):
                        print('now time : ', datetime.now())
                        rewards.append(np.mean(evaluation_reward))
                        episodes.append(e)
                        pylab.plot(episodes, rewards, 'b')
                        pylab.savefig("./save_graph/" + name + "_ppo.png")
                        torch.save(agent.policy_net, "./save_model/" + name + "_ppo")

                        if np.mean(evaluation_reward) > max_eval:
                            torch.save(agent.policy_net.state_dict(), "./save_model/"  + name + "_ppo_best")
                            max_eval = float(np.mean(evaluation_reward))
                            reset_count = 0
                        elif e > 5000:
                            reset_count += 1
                            """
                            if (reset_count == reset_max):
                                print("Training went nowhere, starting again at best model")
                                agent.policy_net.load_state_dict(torch.load("./save_model/spaceinvaders_ppo_best"))
                                agent.update_target_net()
                                reset_count = 0
                            """
                    e += 1
                    evaluation_reward.append(env.score)
                    print("episode:", e, "  score:", env.score,  " epsilon:", agent.epsilon, "   steps:", step,
                      " evaluation reward:", np.mean(evaluation_reward))

                    env.done = False
                    env.score = 0
                    env.history = np.zeros([HISTORY_SIZE+1,84,84], dtype=np.uint8)
                    env.state = env.reset()
                    env.life = number_lives
                    get_init_state(env.history, env.state)
            
        agent.train_policy_net(frame, frame_next_vals)
        agent.update_target_net()
    print("FINISHED TRAINING FOR %s" % (name))
    pylab.figure()
    
    for i in range(len(envs)):
        envs[i]._env.close()
    del envs




 ------- STARTING TRAINING FOR Breakout-v0 ------- 





/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Determing min/max rewards of environment
Min: 0. Max: 4.


/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/model.py:45: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(x[:,:self.action_size] - torch.max(x[:,:self.action_size],1)[0].unsqueeze(1))


Training network. lr: 0.000250. clip: 0.100000


...

Iteration 9148: Policy loss: 0.015440. Value loss: 0.653621. Entropy: 0.976096.
Iteration 9149: Policy loss: 0.005638. Value loss: 0.333117. Entropy: 0.941772.
Iteration 9150: Policy loss: -0.004808. Value loss: 0.220090. Entropy: 0.950506.
Training network. lr: 0.000180. clip: 0.071900
Iteration 9151: Policy loss: 0.009968. Value loss: 0.495907. Entropy: 0.914529.
Iteration 9152: Policy loss: -0.003077. Value loss: 0.247498. Entropy: 0.881938.
Iteration 9153: Policy loss: -0.014066. Value loss: 0.157520. Entropy: 0.895407.
episode: 3733   score: 38.0  epsilon: 1.0    steps: 984  evaluation reward: 24.95
Training network. lr: 0.000180. clip: 0.071900
Iteration 9154: Policy loss: 0.015628. Value loss: 0.609226. Entropy: 0.941573.
Iteration 9155: Policy loss: -0.006042. Value loss: 0.232884. Entropy: 0.908744.
Iteration 9156: Policy loss: -0.009802. Value loss: 0.113023. Entropy: 0.912757.
episode: 3734   score: 33.0  epsilon: 1.0    steps: 48  evaluation reward: 25.09
Training network. 

Iteration 9217: Policy loss: 0.020797. Value loss: 0.708341. Entropy: 0.901199.
Iteration 9218: Policy loss: 0.008367. Value loss: 0.301321. Entropy: 0.881976.
Iteration 9219: Policy loss: -0.006166. Value loss: 0.158563. Entropy: 0.906270.
episode: 3752   score: 20.0  epsilon: 1.0    steps: 504  evaluation reward: 25.55
Training network. lr: 0.000179. clip: 0.071744
Iteration 9220: Policy loss: 0.020154. Value loss: 1.052219. Entropy: 0.880938.
Iteration 9221: Policy loss: 0.006980. Value loss: 0.351531. Entropy: 0.885071.
Iteration 9222: Policy loss: -0.008236. Value loss: 0.186022. Entropy: 0.890897.
episode: 3753   score: 46.0  epsilon: 1.0    steps: 176  evaluation reward: 25.71
episode: 3754   score: 16.0  epsilon: 1.0    steps: 184  evaluation reward: 25.53
Training network. lr: 0.000179. clip: 0.071744
Iteration 9223: Policy loss: 0.020531. Value loss: 0.625617. Entropy: 0.807609.
Iteration 9224: Policy loss: 0.003422. Value loss: 0.277763. Entropy: 0.815939.
Iteration 9225: Po

Iteration 9284: Policy loss: 0.000593. Value loss: 0.321943. Entropy: 0.826326.
Iteration 9285: Policy loss: -0.004180. Value loss: 0.223631. Entropy: 0.814487.
Training network. lr: 0.000179. clip: 0.071587
Iteration 9286: Policy loss: 0.019775. Value loss: 0.691700. Entropy: 0.864542.
Iteration 9287: Policy loss: 0.002682. Value loss: 0.283014. Entropy: 0.852793.
Iteration 9288: Policy loss: -0.012438. Value loss: 0.137451. Entropy: 0.862979.
episode: 3774   score: 37.0  epsilon: 1.0    steps: 760  evaluation reward: 26.06
Training network. lr: 0.000179. clip: 0.071587
Iteration 9289: Policy loss: 0.008940. Value loss: 0.397307. Entropy: 0.903358.
Iteration 9290: Policy loss: -0.008911. Value loss: 0.154674. Entropy: 0.905209.
Iteration 9291: Policy loss: -0.015222. Value loss: 0.088323. Entropy: 0.910098.
Training network. lr: 0.000179. clip: 0.071587
Iteration 9292: Policy loss: 0.009914. Value loss: 0.891661. Entropy: 0.873244.
Iteration 9293: Policy loss: -0.002304. Value loss: 0

episode: 3792   score: 18.0  epsilon: 1.0    steps: 384  evaluation reward: 26.82
Training network. lr: 0.000178. clip: 0.071283
Iteration 9355: Policy loss: 0.003439. Value loss: 0.566411. Entropy: 0.966028.
Iteration 9356: Policy loss: -0.004150. Value loss: 0.274021. Entropy: 0.973757.
Iteration 9357: Policy loss: -0.012554. Value loss: 0.170054. Entropy: 0.968309.
episode: 3793   score: 30.0  epsilon: 1.0    steps: 424  evaluation reward: 26.91
Training network. lr: 0.000178. clip: 0.071283
Iteration 9358: Policy loss: 0.009030. Value loss: 0.804761. Entropy: 0.920980.
Iteration 9359: Policy loss: -0.001625. Value loss: 0.326817. Entropy: 0.916075.
Iteration 9360: Policy loss: -0.008229. Value loss: 0.145830. Entropy: 0.908789.
episode: 3794   score: 23.0  epsilon: 1.0    steps: 408  evaluation reward: 26.99
Training network. lr: 0.000178. clip: 0.071283
Iteration 9361: Policy loss: 0.010843. Value loss: 0.693245. Entropy: 0.854609.
Iteration 9362: Policy loss: -0.002840. Value los

Iteration 9422: Policy loss: -0.000544. Value loss: 0.323217. Entropy: 0.946247.
Iteration 9423: Policy loss: -0.001883. Value loss: 0.193364. Entropy: 0.951093.
episode: 3813   score: 31.0  epsilon: 1.0    steps: 952  evaluation reward: 27.52
Training network. lr: 0.000178. clip: 0.071126
Iteration 9424: Policy loss: 0.017955. Value loss: 0.637136. Entropy: 0.894556.
Iteration 9425: Policy loss: 0.009436. Value loss: 0.293663. Entropy: 0.904422.
Iteration 9426: Policy loss: -0.006884. Value loss: 0.172581. Entropy: 0.900164.
Training network. lr: 0.000178. clip: 0.071126
Iteration 9427: Policy loss: 0.005201. Value loss: 0.347610. Entropy: 0.945189.
Iteration 9428: Policy loss: -0.006746. Value loss: 0.185642. Entropy: 0.940770.
Iteration 9429: Policy loss: -0.016055. Value loss: 0.127150. Entropy: 0.937647.
Training network. lr: 0.000178. clip: 0.071126
Iteration 9430: Policy loss: 0.011788. Value loss: 0.276701. Entropy: 0.913673.
Iteration 9431: Policy loss: 0.001249. Value loss: 0

In [2]:
def test_best(name):
    env = GameEnv(name)
    print("\n\n\n ------- TESTING BEST MODEL FOR %s ------- \n\n\n" % (name))
    number_lives = env.life
    
    if (name == 'SpaceInvaders-v0'):
        action_size = 4
    else:
        action_size = env.action_space.n
    rewards, episodes = [], []
    
    e = 0
    frame = 0

    agent = Agent(action_size)
    agent.policy_net.load_state_dict(torch.load("./save_model/" + name + "_ppo_best"))
    agent.update_target_net()
    agent.policy_net.eval()
    evaluation_reward = deque(maxlen=evaluation_reward_length)

    for i in range(100):
        env.done = False
        env.score = 0
        env.history = np.zeros([HISTORY_SIZE+1,84,84], dtype=np.uint8)
        env.state = env.reset()
        env.life = number_lives
        get_init_state(env.history, env.state)
        step = 0
        while not env.done:
            curr_state = env.history[HISTORY_SIZE-1,:,:]
            net_in = env.history[:HISTORY_SIZE,:,:]
            action, value = agent.get_action(np.float32(net_in) / 255.)
            
            next_state, env.reward, env.done, env.info = env.step(action)
            env._env.render()
            
            frame_next_state = get_frame(next_state)
            
            env.history[HISTORY_SIZE,:,:] = frame_next_state
            terminal_state = check_live(env.life, env.info['ale.lives'])
            env.life = env.info['ale.lives']
            
            
            env.score += env.reward
            env.history[:HISTORY_SIZE, :, :] = env.history[1:,:,:]
            step += 1
        

        evaluation_reward.append(env.score)
        print("episode:", e, "  score:", env.score,  " epsilon:", agent.epsilon, "   steps:", step,
                      " evaluation reward:", np.mean(evaluation_reward))
            

In [ ]:
test_best('MsPacman-v0')

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/model.py:45: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(x[:,:self.action_size] - torch.max(x[:,:self.action_size],1)[0].unsqueeze(1))





 ------- TESTING BEST MODEL FOR MsPacman-v0 ------- 



episode: 0   score: 1270.0  epsilon: 1.0    steps: 965  evaluation reward: 1270.0
episode: 0   score: 1450.0  epsilon: 1.0    steps: 1162  evaluation reward: 1360.0
episode: 0   score: 1270.0  epsilon: 1.0    steps: 746  evaluation reward: 1330.0
episode: 0   score: 1790.0  epsilon: 1.0    steps: 1058  evaluation reward: 1445.0
episode: 0   score: 1160.0  epsilon: 1.0    steps: 1112  evaluation reward: 1388.0
episode: 0   score: 1350.0  epsilon: 1.0    steps: 1073  evaluation reward: 1381.6666666666667
episode: 0   score: 1500.0  epsilon: 1.0    steps: 1265  evaluation reward: 1398.5714285714287
episode: 0   score: 1270.0  epsilon: 1.0    steps: 960  evaluation reward: 1382.5
episode: 0   score: 1670.0  epsilon: 1.0    steps: 1405  evaluation reward: 1414.4444444444443
episode: 0   score: 1230.0  epsilon: 1.0    steps: 1457  evaluation reward: 1396.0
episode: 0   score: 1600.0  epsilon: 1.0    steps: 1223  evaluation reward: 14